In [5]:
import pandas as pd
import numpy as np


df = pd.read_csv('Data_Crash Risk.csv', parse_dates=['Dates', 'Date'], infer_datetime_format=True, low_memory=False)


df['Nifty50 prices'] = df['Nifty50 prices'].replace({',': ''}, regex=True).astype(float)

# Function to calculate residual return (W)
def calculate_residual_return(row):
    r = row['Price']
    
    
    a = b1 = b2 = b3 = b4 = b5 = 0
    
    # Calculating the residual return using the given formula
    residual_return = r - (a + b1 * df['Nifty50 prices'].shift(2).loc[df['Dates'] == row['Dates']].values +
                          b2 * df['Nifty50 prices'].shift(1).loc[df['Dates'] == row['Dates']].values +
                          b3 * df['Nifty50 prices'].loc[df['Dates'] == row['Dates']].values +
                          b4 * df['Nifty50 prices'].shift(-1).loc[df['Dates'] == row['Dates']].values +
                          b5 * df['Nifty50 prices'].shift(-2).loc[df['Dates'] == row['Dates']].values)
    
    return np.log(1 + residual_return)


df['W'] = df.apply(calculate_residual_return, axis=1)

# Grouping by firm and fiscal year
grouped_df = df.groupby(['firm', df['Dates'].dt.year])

# Calculating the third moment and square of the second moment for each group
third_moment = grouped_df['W'].apply(lambda x: (x**3).mean())
second_moment_squared = grouped_df['W'].apply(lambda x: (x**2).mean())**2

# Calculating NCSKEW using the provided formula, handling division by zero
ncskew = np.where(second_moment_squared == 0, np.nan, - (grouped_df.size() * (grouped_df.size() - 1)**(3/2) * third_moment) / ((grouped_df.size() - 1) * (grouped_df.size() - 2) * second_moment_squared**(3/2)))

# Creating a new DataFrame with firm, Dates, and NCSKEW
ncskew_df = pd.DataFrame({
    'firm': grouped_df.apply(lambda x: x['firm'].iloc[0]).values,
    'Dates': grouped_df.apply(lambda x: x['Dates'].iloc[0]).values,
    'NCSKEW': ncskew
})


ncskew_df.to_csv('ncskew_output.csv', index=False)


print(ncskew_df)


C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '13-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '14-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '17-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '18-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\

          firm      Dates                  NCSKEW
0   3IINFOTECH 2005-04-21                   [nan]
1   3IINFOTECH 2006-01-02   [-0.1819068980923445]
2   3IINFOTECH 2007-01-01   [-0.1345966806076909]
3   3IINFOTECH 2008-01-01   [-0.1802071434079155]
4   3IINFOTECH 2009-01-01   [-0.2341240258828197]
5   3IINFOTECH 2010-01-01  [-0.21324754645683083]
6   3IINFOTECH 2011-01-03   [-0.3529932217494273]
7   3IINFOTECH 2012-01-02   [-1.0390358775161832]
8   3IINFOTECH 2013-01-01    [-2.362269085331543]
9   3IINFOTECH 2014-01-01   [-1.4602126545845049]
10  3IINFOTECH 2015-01-01    [-2.868849304607119]
11  3IINFOTECH 2016-01-01    [-2.846529969847633]
12  3IINFOTECH 2017-01-02     [-3.14053382120245]
13  3IINFOTECH 2018-01-01    [-3.351292401029471]
14  3IINFOTECH 2019-01-01    [-6.901083769604531]
15  3IINFOTECH 2020-01-01    [-5.163674731405494]
16        ABAN 2000-01-04   [-3.1842438605294214]
17        ABAN 2001-01-01   [-1.6623608792730866]
18        ABAN 2002-01-01   [-0.6788437469015819]


In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('Data_Crash Risk.csv', parse_dates=['Dates', 'Date'], infer_datetime_format=True, low_memory=False)

# Convert 'Nifty50 prices' column to float
df['Nifty50 prices'] = df['Nifty50 prices'].replace({',': ''}, regex=True).astype(float)

# Function to calculate residual return (W)
def calculate_residual_return(row):
    r = row['Price']
    
    a = b1 = b2 = b3 = b4 = b5 = 0
    
    # Calculating the residual return using the given formula
    residual_return = r - (a + b1 * df['Nifty50 prices'].shift(2).loc[df['Dates'] == row['Dates']].values +
                          b2 * df['Nifty50 prices'].shift(1).loc[df['Dates'] == row['Dates']].values +
                          b3 * df['Nifty50 prices'].loc[df['Dates'] == row['Dates']].values +
                          b4 * df['Nifty50 prices'].shift(-1).loc[df['Dates'] == row['Dates']].values +
                          b5 * df['Nifty50 prices'].shift(-2).loc[df['Dates'] == row['Dates']].values)
    
    return np.log(1 + residual_return)

# Calculate residual returns (W)
df['W'] = df.apply(calculate_residual_return, axis=1)

# Group by firm and fiscal year
grouped_df = df.groupby(['firm', df['Dates'].dt.year])

# Calculate the third moment (kurtosis)
third_moment = grouped_df['W'].apply(lambda x: (x**3).mean())

# Calculate the second moment (volatility)
second_moment = grouped_df['W'].apply(lambda x: (x**2).mean())

# Calculate NCSKEW
ncskew = np.where(second_moment == 0, np.nan, - (grouped_df.size() * (grouped_df.size() - 1)**(3/2) * third_moment) / ((grouped_df.size() - 1) * (grouped_df.size() - 2) * second_moment**(3/2)))

# Calculate other parameters (beta, Sharpe ratio, average daily returns, maximum drawdown, turnover ratio)

# Creating a new DataFrame with all computed parameters
result_df = pd.DataFrame({
    'firm': grouped_df.apply(lambda x: x['firm'].iloc[0]).values,
    'Dates': grouped_df.apply(lambda x: x['Dates'].iloc[0]).values,
    'NCSKEW': ncskew,
    # Add other parameters here
})

# Save the results to a CSV file
result_df.to_csv('financial_parameters_output.csv', index=False)

print(result_df)


C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '13-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '14-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '17-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\pandas\io\parsers\base_parser.py:1070: UserWarning: Parsing '18-01-2000' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
C:\Users\HP\anaconda3\lib\site-packages\

          firm      Dates                 NCSKEW
0   3IINFOTECH 2005-04-21                  [nan]
1   3IINFOTECH 2006-01-02  [-16.228783063884055]
2   3IINFOTECH 2007-01-01   [-16.25805566805695]
3   3IINFOTECH 2008-01-01    [-16.5058009141375]
4   3IINFOTECH 2009-01-01  [-16.484859756374956]
5   3IINFOTECH 2010-01-01  [-16.272362238521342]
6   3IINFOTECH 2011-01-03  [-16.463869861920436]
7   3IINFOTECH 2012-01-02   [-16.57118612612897]
8   3IINFOTECH 2013-01-01  [-16.449988473575864]
9   3IINFOTECH 2014-01-01  [-16.329249615307187]
10  3IINFOTECH 2015-01-01   [-16.97004508292989]
11  3IINFOTECH 2016-01-01  [-16.360132213862922]
12  3IINFOTECH 2017-01-02  [-16.357799726209265]
13  3IINFOTECH 2018-01-01  [-16.646076224684442]
14  3IINFOTECH 2019-01-01   [-16.80899525634684]
15  3IINFOTECH 2020-01-01   [-7.753695501501021]
16        ABAN 2000-01-04  [-16.446077306945384]
17        ABAN 2001-01-01  [-16.287949647403526]
18        ABAN 2002-01-01  [-16.733330511294188]
19        ABAN 2003-